In [1]:
import json, requests, os
import io
from ta1viewer import display_json
GPT_KEY = os.environ.get('OPENAI_API_KEY')
assert GPT_KEY, "Please set the OPENAI_API_KEY environment variable"
API_ROOT = "http://localhost:8000/"

In [2]:
def skema_integrated_pdf_extractions(pdf_path):
    import os
    pdf_path = os.path.expanduser(pdf_path)
    assert os.path.exists(pdf_path)

    URL= "https://api.askem.lum.ai"

    params = {
       "annotate_skema":True,
       "annotate_mit": True
    }

    files = [("pdfs", (str(pdf_path), open(str(pdf_path), "rb")))]
    response = requests.post(f"{URL}/text-reading/integrated-pdf-extractions", params=params, files=files)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        raise  Exception(f'{response.status_code=} {response.text=}')

In [3]:
data = json.load(open('/Users/orm/Downloads/Patty-reviewed-1-28-24-bertozzi-et-al-2020-the-challenges-of-modeling-and-forecasting-the-spread-of-covid-19.json'))

In [18]:
extractions=data['outputs'][0]['data']

In [48]:
from mitaskem.api.routers.annotation import AttributeCollection

In [50]:
att = AttributeCollection.from_json(extractions)

In [57]:
#att.attributes

In [88]:
import jmespath as jp
import pandas as pd
var_extractor_expr = """
        outputs[0].data.attributes[? type == 'anchored_entity'][].payload
            .{ id:id.id,
                names:mentions[].name,
                values:value_descriptions[].value.amount
        }
    """
pd.DataFrame(jp.search(var_extractor_expr.replace('\n', ' '), data))

,id,names,values
0,R:1407870398,[dDepartment of Computer Science],[90095]
1,R:-2133161548,[fDepartment of Political Science],[30332]
2,E:-443623759,[Tx],[]
3,R:889375648,[Mechanical],[90095]
4,R:1807253537,[eDepartment of Mathematics],[46202]
...,...,...,...
165,mit25,[- r],[R/N]
166,mit26,[-],[0 < << 1]
167,mit27,[- λI(t)],None
168,mit28,[- S(t)],None


In [84]:
var_extractor_expr = "attributes[? type == 'anchored_entity'].payload"
jp.search(var_extractor_expr, extractions)[0].keys()

dict_keys(['id', 'mentions', 'text_descriptions', 'value_descriptions', 'groundings', 'data_columns'])

In [100]:
## send post request to
params = {
    "gpt_key": GPT_KEY,
}

files = [("extractions_file", ('', io.BytesIO(json.dumps(data).encode('utf-8'))))]

response = requests.post('http://localhost:8000/annotation/list_scenarios/', params=params, files=files)
response.status_code, print(response.text)

[{"location":"17 March 2020","entities":[{"id":"R:1330896514","names":["NPIs"],"values":[]}]},{"location":"AIC","entities":[{"id":"E:-27420438","names":["a"],"values":[]},{"id":"E:1221701905","names":["SIR"],"values":[]},{"id":"E:-529909366","names":["SIR"],"values":[]},{"id":"E:1242757458","names":["SIR"],"values":[]},{"id":"R:1326130415","names":["AIC"],"values":[]},{"id":"E:587018829","names":["AICmin"],"values":["2"]},{"id":"R:166630294","names":["AIC"],"values":[]},{"id":"E:639721417","names":["CA"],"values":[]},{"id":"R:-418568586","names":["NY"],"values":[]},{"id":"R:1826066952","names":["k"],"values":[]},{"id":"R:719599970","names":["b"],"values":[]}]},{"location":"Aerospace Engineering","entities":[{"id":"R:1407870398","names":["dDepartment of Computer Science"],"values":["90095"]},{"id":"R:-2133161548","names":["fDepartment of Political Science"],"values":["30332"]},{"id":"E:-443623759","names":["Tx"],"values":[]},{"id":"R:889375648","names":["Mechanical"],"values":["90095"]}

(200, None)

In [101]:
response.json()

[{'location': '17 March 2020',
  'entities': [{'id': 'R:1330896514', 'names': ['NPIs'], 'values': []}]},
 {'location': 'AIC',
  'entities': [{'id': 'E:-27420438', 'names': ['a'], 'values': []},
   {'id': 'E:1221701905', 'names': ['SIR'], 'values': []},
   {'id': 'E:-529909366', 'names': ['SIR'], 'values': []},
   {'id': 'E:1242757458', 'names': ['SIR'], 'values': []},
   {'id': 'R:1326130415', 'names': ['AIC'], 'values': []},
   {'id': 'E:587018829', 'names': ['AICmin'], 'values': ['2']},
   {'id': 'R:166630294', 'names': ['AIC'], 'values': []},
   {'id': 'E:639721417', 'names': ['CA'], 'values': []},
   {'id': 'R:-418568586', 'names': ['NY'], 'values': []},
   {'id': 'R:1826066952', 'names': ['k'], 'values': []},
   {'id': 'R:719599970', 'names': ['b'], 'values': []}]},
 {'location': 'Aerospace Engineering',
  'entities': [{'id': 'R:1407870398',
    'names': ['dDepartment of Computer Science'],
    'values': ['90095']},
   {'id': 'R:-2133161548',
    'names': ['fDepartment of Politica